### Importing all the libraries

In [17]:
!conda install -c conda-forge geocoder
!conda install -c conda-forge geopy
!conda install -c anaconda scikit-learn
!conda install -c anaconda beautifulsoup4 
!conda install -c conda-forge folium 

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geocoder:   1.38.1-py_0  conda-forge
    orderedset: 2.0-py35_0   conda-forge
    ratelim:    0.1.6-py35_0 conda-forge

orderedset-2.0 100% |################################| Time: 0:00:00  13.64 MB/s
ratelim-0.1.6- 100% |################################| Time: 0:00:00  10.50 MB/s
geocoder-1.38. 100% |################################| Time: 0:00:00  33.32 MB/s


## Installing all the packages required

## Importing all the libraries

In [1]:
import pandas as pd
import requests
import numpy as np
import folium
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup

# #Getting table from Wikipedia

In [2]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

My_table = soup.find('table',{'class':'wikitable sortable'})
links = My_table.findAll('tr')[1:]
Postcode = []
Borough = []
Neighborhood = []
for link in links:
    x = link.find_all('td')[0]
    y = link.find_all('td')[2]
    Postcode.append(x.text)
    Borough.append(y.text)
    z = link.find_all('td')[1]
    Neighborhood.append(z.text)

## Creating a data frame from the table

In [3]:
df = pd.DataFrame(columns=['Postcode','Borough','Neighborhood'])

## Filling the pandas data frame

In [4]:
df['Postcode'] = Postcode
df['Borough'] = Neighborhood
df['Neighborhood'] = Borough

## cleaning the neighborhood coloumn

In [5]:
x = df['Neighborhood']
y = list(x)
z = [col.split('\n')[0] for col in y]
df['Neighborhood'] = z

## Removing all NaN values

In [6]:
df.replace('Not assigned','NaN', inplace = True)
borough_not_required = df[df['Borough']== 'NaN'].index.tolist()
df = df.drop(borough_not_required).reset_index(drop=True)
df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


## Joining the Neighborhood coloumn for the same Postcode

In [7]:
x  = df.groupby(['Postcode','Borough'])['Neighborhood'].apply(','.join)
toronto = pd.DataFrame(x) 
toronto.reset_index(inplace=True)
toronto.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
toronto.shape

(103, 3)

## Task 2

In [9]:
import geocoder

## Defining function for getting latitude longitude for postal code

In [10]:
def getlatlng(postal_code):
    lat_lng_coords = None
    while (lat_lng_coords == None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        return lat_lng_coords

getlatlng('M1B')
coords = [getlatlng(x) for x in toronto['Postcode'].tolist()]

## Adding coordinates to main dataframe i.e. toronto

In [11]:
toronto['Latitude']=[coords[i][0] for i in range(len(coords))]
toronto['Longitude'] = [coords[i][1] for i in range(len(coords))] 

In [12]:
toronto.head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.726245,-79.263670
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.713133,-79.285055
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.723575,-79.234976
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.696665,-79.260163


In [13]:
toronto[toronto['Postcode'] == 'M5G']

,Postcode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.656091,-79.38493
